In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import re
import warnings
warnings.filterwarnings('ignore')
import cmc_parser as cp


# Individual file parsing

In [2]:
def parse_bhformation_dat(model_path):

    filepath= model_path+'/initial.bhformation.dat'
    # Define column names for the core data we want to keep
    column_names = [
        'time', 'r', 'binary', 'ID', 'zams_m', 'm_progenitor', 'bh_mass',
        'bh_spin', 'birth_kick'
    ]

    # Read the file, skipping any lines that start with #
    data = []
    with open(filepath, 'r') as file:
        for line in file:
            if not line.startswith('#'):
                # Split the line by whitespace
                values = line.strip().split()
                if len(values) >= 9:  # Ensure we have enough values for our columns
                    # Only take the first 9 values (excluding vs columns)
                    data.append(values[:9])

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=column_names)

    # Convert columns to appropriate data types
    numeric_cols = ['time', 'r', 'zams_m', 'm_progenitor', 'bh_mass', 'bh_spin']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])

    # Convert binary and ID to integers
    df['binary'] = df['binary'].astype(int)
    df['ID'] = df['ID'].astype(int)

    # Handle birth_kick column
    df['birth_kick'] = pd.to_numeric(df['birth_kick'], errors='coerce')

    return df
parse_bhformation_dat('data/N2.0rv0.5rg2.0Z0.0002')

,time,r,binary,ID,zams_m,m_progenitor,bh_mass,bh_spin,birth_kick
0,0.009197,0.190420,1,124149,0.000000,47.24320,42.51890,0,0.0000
1,0.009476,0.784866,0,178115,104.318000,101.11100,40.50000,0,1.0000
2,0.009920,0.069105,1,150155,0.000000,37.76810,33.99130,0,0.0000
3,0.010120,0.109359,1,228329,0.000000,35.36190,31.82570,0,0.0000
4,0.010364,1.755900,0,9114,0.181263,32.65030,29.38530,0,1.0000
...,...,...,...,...,...,...,...,...,...
338,0.039135,0.127507,1,216993,0.000000,75.53730,67.98360,0,0.0000
339,0.040909,0.233458,0,185677,0.178679,25.05900,9.83723,0,151.7321
340,0.043852,0.321977,0,238752,0.336246,21.79050,19.61150,0,1.0000
341,6.507828,0.035967,1,0,0.000000,4.23882,16.73940,0,0.0000


In [3]:
def parse_collision_log(model_path):


    filepath = model_path + '/initial.collision.log'
    data = {
        'time': [],
        'collision_type': [],
        'idm': [],
        'mm': [],
        'id1': [],
        'm1': [],
        'id2': [],
        'm2': [],
        'r': [],
        'typem': [],
        'type1': [],
        'type2': []
    }

    def add_collision_record(time, collision_type, idm, mm,
                             id1, m1, id2, m2,
                             r, typem, type1, type2):
        data['time'].append(time)
        data['collision_type'].append(collision_type)
        data['idm'].append(idm)
        data['mm'].append(mm)
        data['id1'].append(id1)
        data['m1'].append(m1)
        data['id2'].append(id2)
        data['m2'].append(m2)
        data['r'].append(r)
        data['typem'].append(typem)
        data['type1'].append(type1)
        data['type2'].append(type2)

    with open(filepath, 'r') as file:
        for line in file:
            if not line.startswith('t='):
                continue

            # common to all
            time_match   = re.search(r't=([\d.eE+-]+)', line)
            type_match   = re.search(r'\s(binary-[a-z]+|single-single)\s', line)
            idm_mm_match = re.search(r'idm=(\d+)\(mm=([\d.eE+-]+)\)', line)
            r_match      = re.search(r'\(r=([\d.eE+-]+)\)', line)
            typem_match  = re.search(r'typem=(\d+)', line)

            # skip weird lines 
            if not all([time_match, type_match, idm_mm_match,
                        r_match, typem_match]):
                continue

            time       = float(time_match.group(1))
            collision_type = type_match.group(1)
            original_idm   = int(idm_mm_match.group(1))
            original_mm    = float(idm_mm_match.group(2))
            r              = float(r_match.group(1))
            typem          = int(typem_match.group(1))

            idm_block   = re.findall(r'id(\d)=(\d+)\(m\1=([\d.eE+-]+)\)', line)
            type_block  = re.findall(r'type(\d)=(\d+)', line)

            bodies = [(int(idx), int(star_id), float(mass))
                      for idx, star_id, mass in idm_block]
            type_dict = {int(idx): int(t) for idx, t in type_block}

            # need at least a binary encounter
            if len(bodies) < 2:
                continue

            # sort by mass (descending) while preserving idx for type lookup
            bodies_sorted = sorted(
                [(star_id, mass, type_dict[idx])
                 for idx, star_id, mass in bodies],
                key=lambda x: x[1], reverse=True)

            # ----------------------------------------------------------------
            # 2-body collision  → write one row unchanged
            # ----------------------------------------------------------------
            if len(bodies_sorted) == 2:
                (id1, m1, t1), (id2, m2, t2) = bodies_sorted
                add_collision_record(
                    time=time,
                    collision_type=collision_type,
                    idm=original_idm,
                    mm=original_mm,
                    id1=id1, m1=m1,
                    id2=id2, m2=m2,
                    r=r,
                    typem=typem,
                    type1=t1, type2=t2
                )
                continue

            # ----------------------------------------------------------------
            # 3-body collision  → two rows (big+mid, product+small)
            # ----------------------------------------------------------------
            if len(bodies_sorted) == 3:
                (big_id, big_m, big_t), \
                (mid_id, mid_m, mid_t), \
                (sml_id, sml_m, sml_t) = bodies_sorted

                # Collision 1
                cid1 = big_id + mid_id
                cm1  = big_m  + mid_m
                add_collision_record(time, collision_type,
                                     cid1, cm1,
                                     big_id, big_m,
                                     mid_id, mid_m,
                                     r, typem,
                                     big_t, mid_t)

                # Collision 2  (final product keeps original idm/mm)
                add_collision_record(time, collision_type,
                                     original_idm, original_mm,
                                     cid1, cm1,
                                     sml_id, sml_m,
                                     r, typem,
                                     big_t,       sml_t) # NOTE I CHANGED THE -1 HERE TO BIG_T
                continue

            # ----------------------------------------------------------------
            # 4-body collision (two binaries)  → three rows
            # scheme: (big1+big2) → (prod+small1) → (prod+small2)
            # ----------------------------------------------------------------
            if len(bodies_sorted) == 4:
                (b1_id, b1_m, b1_t), \
                (b2_id, b2_m, b2_t), \
                (s1_id, s1_m, s1_t), \
                (s2_id, s2_m, s2_t) = bodies_sorted

                # Collision 1: two big stars
                cid1 = b1_id + b2_id
                cm1  = b1_m  + b2_m
                add_collision_record(time, collision_type,
                                     cid1, cm1,
                                     b1_id, b1_m,
                                     b2_id, b2_m,
                                     r, typem,
                                     b1_t, b2_t)

                # Collision 2: product + first small star
                cid2 = cid1 + s1_id
                cm2  = cm1  + s1_m
                add_collision_record(time, collision_type,
                                     cid2, cm2,
                                     cid1, cm1,
                                     s1_id, s1_m,
                                     r, typem,
                                     -1,     s1_t)

                # Collision 3: product + second small star (final)
                add_collision_record(time, collision_type,
                                     original_idm, original_mm,
                                     cid2, cm2,
                                     s2_id, s2_m,
                                     r, typem,
                                     -1,     s2_t)
                continue


            raise ValueError(
                f"Encounter with {len(bodies_sorted)} participants "
            )

    df = pd.DataFrame(data)
    return df


In [4]:
def parse_merger_file(model_path):
    filepath = model_path + '/initial.semergedisrupt.log'
    # Initialize lists to store the extracted data
    data = {
        'time': [],
        'interaction_type': [],
        'id_rem': [],
        'mass_rem': [],
        'id1': [],
        'm1': [],
        'id2': [],
        'm2': [],
        'r': [],
        'type_rem': [],
        'type1': [],
        'type2': []
    }

    with open(filepath, 'r') as file:
        for line in file:
            # Skip comment lines
            if line.startswith('#'):
                continue

            # Check if this is a data line
            if line.startswith('t='):
                # Extract time
                time_match = re.search(r't=([0-9.eE+-]+)', line)
                time = float(time_match.group(1)) if time_match else np.nan

                # Extract interaction type
                type_match = re.search(r't=[\d.eE+-]+\s+(\w+)', line)
                interaction_type = type_match.group(1) if type_match else np.nan

                # Extract remnant ID and mass
                id_rem_match = re.search(r'idr=(\d+)\(mr=([\d.eE+-]+)\)', line)
                id_rem = int(id_rem_match.group(1)) if id_rem_match else np.nan
                mass_rem = float(id_rem_match.group(2)) if id_rem_match else np.nan

                # Extract id1 and m1
                id1_m1_match = re.search(r'id1=(\d+)\(m1=([\d.eE+-]+)\)', line)
                id1 = int(id1_m1_match.group(1)) if id1_m1_match else np.nan
                m1 = float(id1_m1_match.group(2)) if id1_m1_match else np.nan

                # Extract id2 and m2
                id2_m2_match = re.search(r'id2=(\d+)\(m2=([\d.eE+-]+)\)', line)
                id2 = int(id2_m2_match.group(1)) if id2_m2_match else np.nan
                m2 = float(id2_m2_match.group(2)) if id2_m2_match else np.nan

                # Extract radius
                r_match = re.search(r'\(r=([\d.eE+-]+)\)', line)
                r = float(r_match.group(1)) if r_match else np.nan

                # Extract types
                type_rem_match = re.search(r'typer=(\d+)', line)
                type1_match = re.search(r'type1=(\d+)', line)
                type2_match = re.search(r'type2=(\d+)', line)

                type_rem = int(type_rem_match.group(1)) if type_rem_match else np.nan
                type1 = int(type1_match.group(1)) if type1_match else np.nan
                type2 = int(type2_match.group(1)) if type2_match else np.nan

                # Append data
                data['time'].append(time)
                data['interaction_type'].append(interaction_type)
                data['id_rem'].append(id_rem)
                data['mass_rem'].append(mass_rem)
                data['id1'].append(id1)
                data['m1'].append(m1)
                data['id2'].append(id2)
                data['m2'].append(m2)
                data['r'].append(r)
                data['type_rem'].append(type_rem)
                data['type1'].append(type1)
                data['type2'].append(type2)

    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Usage
merger_df = parse_merger_file('data/N2.0rv0.5rg2.0Z0.0002')

merger_df


,time,interaction_type,id_rem,mass_rem,id1,m1,id2,m2,r,type_rem,type1,type2
0,0.001989,disrupt1,312311.0,75.21870,312311,74.964600,162670,0.254204,0.314027,1.0,1,0
1,0.005335,disrupt1,136651.0,89.76560,136651,89.665800,125486,0.099913,0.221251,1.0,1,0
2,0.005919,disrupt1,161709.0,83.86030,161709,78.209600,204653,5.650770,0.063689,1.0,1,1
3,0.007645,disrupt1,325918.0,78.02040,325918,77.910400,158314,0.110015,0.062531,1.0,1,0
4,0.008486,disrupt2,175727.0,109.86200,161895,0.092841,175727,109.886000,0.181820,2.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
618,9.037510,disruptboth,NaN,NaN,210009,1.664140,160681,1.102050,0.012559,NaN,8,1
619,9.157400,disrupt2,287355.0,2.35583,155861,0.980142,287355,1.870000,0.017850,5.0,11,6
620,9.162860,disrupt2,378095.0,2.80844,162500,0.876363,378095,2.070240,0.008220,4.0,11,5
621,9.181540,disruptboth,NaN,NaN,219323,1.107870,125797,1.202410,0.011862,NaN,5,11


# parsing bh mergers across models

In [5]:
def parse_merger_files(*filenames):
    """Return a single DataFrame with all rows from the supplied catalogue files."""
    i=0
    # setting up col properties
    PROPERTIES = [
    ("model_num",      float),    # 0
    ("rv",             float),  # 1
    ("rg",             float),    # 2
    ("Z",              float),  # 3
    ("N",              float),    # 4  (×10^5)
    ("merger_time",    float),  # 5  (Myr)
    ("id1",            float),    # 6
    ("id2",            float),    # 7
    ("m1",             float),  # 8  (M⊙)
    ("m2",             float),  # 9
    ("spin1",          float),  # 10
    ("spin2",          float),  # 11
    ("v_kick",         float),  # 12 (km s⁻¹)
    ("v_esc",          float),  # 13
    ("merger_channel", float),    # 14
    ("id3",            float),    # 15 (remnant id)
    ("m3",             float),  # 16
    ("spin3",          float),  # 17
    ("a_final",        float),  # 18 (AU)
    ("e_final",        float),  # 19
    ("e_10hz",         float)   # 20
    ]
    COLS = [c for c, _ in PROPERTIES]
    records = []
    for fname in filenames:
        try:
            with open(fname) as fh:
                for line in fh:
                    if line.startswith("#") or not line.strip():
                        continue
                    parts = line.split()
                    if len(parts) < len(PROPERTIES):
                        print(line)
                        # silently skip malformed lines
                        continue
                    rec = {col: typ(parts[i]) for i, (col, typ) in enumerate(PROPERTIES)}
                    records.append(rec)
        except FileNotFoundError:
            print(f"  File '{fname}' not found – skipped.")
    df = pd.DataFrame.from_records(records, columns=COLS)
    print(i)


    return df

ag_bh_merge = parse_merger_files("merger_files/BBHmergers3.dat", "merger_files/BBHmergers_SSC.dat")

ag_bh_merge # no model paths

0


,model_num,rv,rg,Z,N,merger_time,id1,id2,m1,m2,...,spin2,v_kick,v_esc,merger_channel,id3,m3,spin3,a_final,e_final,e_10hz
0,1.0,0.5,2.0,0.0002,2.0,24.821458,178115.0,368347.0,40.5000,31.578800,...,0.0,69.399800,70.1823,4.0,368348.0,68.6898,0.678055,0.000199,0.788704,-1.000000
1,1.0,0.5,2.0,0.0002,2.0,28.974058,17637.0,216938.0,33.7537,39.331800,...,0.0,43.499500,76.8347,4.0,216939.0,69.5926,0.683270,0.000285,0.266025,-1.000000
2,1.0,0.5,2.0,0.0002,2.0,33.848945,216939.0,217652.0,69.5926,46.634300,...,0.0,886.331000,52.9445,2.0,216939.0,111.2130,0.637824,0.004117,0.163180,-1.000000
3,1.0,0.5,2.0,0.0002,2.0,34.932509,95702.0,298584.0,22.7363,17.729100,...,0.0,69.383200,59.4337,2.0,95702.0,38.5628,0.678059,0.001086,0.587944,-1.000000
4,1.0,0.5,2.0,0.0002,2.0,37.349633,228330.0,253249.0,31.8257,29.710400,...,0.0,19.746800,58.9670,4.0,228332.0,58.5717,0.685814,0.000241,0.258842,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14343,149.0,2.0,20.0,0.0020,1.0,3994.854440,1012119.0,9149421.0,-1.0000,29.966549,...,-1.0,0.000000,0.0000,-1.0,-1.0,1.0000,-1.000000,-1.000000,-1.000000,0.017292
14344,149.0,2.0,20.0,0.0020,1.0,4038.046583,1687518.0,8243294.0,-1.0000,46.622272,...,-1.0,0.683903,0.0000,-1.0,-1.0,1.0000,-1.000000,-1.000000,-1.000000,0.091357
14345,149.0,2.0,20.0,0.0020,1.0,4106.425254,2107286.0,10845792.0,-1.0000,16.656953,...,-1.0,0.000000,0.0000,-1.0,-1.0,1.0000,-1.000000,-1.000000,-1.000000,0.041849
14346,149.0,2.0,20.0,0.0020,1.0,4139.542502,7659600.0,10623895.0,-1.0000,29.087456,...,-1.0,0.000000,0.0000,-1.0,-1.0,1.0000,-1.000000,-1.000000,-1.000000,0.090165


## tagging bbh mergers with model path

In [6]:
def process_dataframe_by_N(df):

    
    # Step 1: Sort dataframe by 'N'
    df_sorted = df.sort_values('N').copy()
    
    # Step 2: Split into two frames
    df_less_than_10 = df_sorted[df_sorted['N'] < 10].copy()
    df_greater_equal_10 = df_sorted[df_sorted['N'] >= 10].copy()
    
    print(f"Rows with N < 10: {len(df_less_than_10)}")
    print(f"Rows with N >= 10: {len(df_greater_equal_10)}")
    
    # Step 3: For N >= 10 table, change N to integer
    if not df_greater_equal_10.empty:
        df_greater_equal_10['N'] = df_greater_equal_10['N'].astype(int)
        print("Converted N >= 10 values to integers")
    
    # Step 4: Apply model_path creation to both dataframes
    # For N < 10 frame
    if not df_less_than_10.empty:
        df_less_than_10['model_path'] = ('data\\N' + 
                                        df_less_than_10['N'].astype(str) + 
                                        'rv' + df_less_than_10['rv'].astype(str) + 
                                        'rg' + df_less_than_10['rg'].astype(str) + 
                                        'Z' + df_less_than_10['Z'].astype(str))
    
    # For N >= 10 frame
    if not df_greater_equal_10.empty:
        df_greater_equal_10['model_path'] = ('data\\N' + 
                                            df_greater_equal_10['N'].astype(str) + 
                                            'rv' + df_greater_equal_10['rv'].astype(str) + 
                                            'rg' + df_greater_equal_10['rg'].astype(str) + 
                                            'Z' + df_greater_equal_10['Z'].astype(str))
    
    # Step 5: Convert N values in second dataframe back to floats
    if not df_greater_equal_10.empty:
        df_greater_equal_10['N'] = df_greater_equal_10['N'].astype(float)
        print("Converted N >= 10 values back to floats")
    
    # Step 6: Combine the two frames again
    df_combined = pd.concat([df_less_than_10, df_greater_equal_10], ignore_index=True)
    
    # Sort by original order if needed, or keep sorted by N
    df_final = df_combined.sort_values('N').reset_index(drop=True)
    
    print(f"Final combined dataframe shape: {df_final.shape}")
    
    return df_final



In [7]:
ag_bh_merge = process_dataframe_by_N(ag_bh_merge)
ag_bh_merge


Rows with N < 10: 5536
Rows with N >= 10: 8812
Converted N >= 10 values to integers
Converted N >= 10 values back to floats
Final combined dataframe shape: (14348, 22)


,model_num,rv,rg,Z,N,merger_time,id1,id2,m1,m2,...,v_kick,v_esc,merger_channel,id3,m3,spin3,a_final,e_final,e_10hz,model_path
0,149.0,2.0,20.0,0.0020,1.0,2144.929793,62090.0,1934343.0,66.97210,72.4762,...,454.037000,129.4600,2.0,62090.0,132.6500,0.682164,0.008331,0.043318,-1.0,data\N1.0rv2.0rg20.0Z0.002
1,149.0,2.0,20.0,0.0020,1.0,2131.156995,7338925.0,1533922.0,16.43480,31.3124,...,150.523000,130.1470,3.0,12033922.0,45.8084,0.632367,0.000128,0.843304,-1.0,data\N1.0rv2.0rg20.0Z0.002
2,149.0,2.0,20.0,0.0020,1.0,2121.769751,10658409.0,92882.0,41.10010,42.0878,...,6.832950,139.8070,5.0,10658409.0,79.1733,0.686383,1.549980,0.999858,-1.0,data\N1.0rv2.0rg20.0Z0.002
3,149.0,2.0,20.0,0.0020,1.0,2113.954663,10720103.0,13016234.0,23.43160,30.0190,...,69.119300,135.5760,3.0,10720105.0,50.9368,0.678127,0.000145,0.821159,-1.0,data\N1.0rv2.0rg20.0Z0.002
4,149.0,2.0,20.0,0.0020,1.0,2113.701948,7209453.0,11690997.0,28.61570,36.9853,...,71.419400,134.4490,5.0,7209453.0,62.5217,0.677527,0.575687,0.999752,-1.0,data\N1.0rv2.0rg20.0Z0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14343,147.0,1.0,20.0,0.0200,32.0,716.737634,1061758.0,2577018.0,14.93740,14.9827,...,0.872944,93.4632,2.0,1061758.0,28.4758,0.686459,0.002783,0.218032,-1.0,data\N32rv1.0rg20.0Z0.02
14344,147.0,1.0,20.0,0.0200,32.0,406.007572,1749200.0,3954687.0,7.63013,12.9594,...,132.329000,112.0790,2.0,1749200.0,19.7060,0.649347,0.378760,0.999481,-1.0,data\N32rv1.0rg20.0Z0.02
14345,147.0,1.0,20.0,0.0200,32.0,405.472287,2514485.0,5593935.0,28.22760,14.8388,...,514.010000,116.0560,2.0,2514485.0,41.6730,0.503973,0.103354,0.992652,-1.0,data\N32rv1.0rg20.0Z0.02
14346,147.0,1.0,20.0,0.0200,32.0,461.040435,1785753.0,3558028.0,14.89870,14.2147,...,13.511400,105.6060,2.0,1785753.0,27.7094,0.686158,0.003220,0.422040,-1.0,data\N32rv1.0rg20.0Z0.02


In [8]:
y = np.array(ag_bh_merge['model_num'].unique())
y

array([149.,  89.,   1.,  13.,  69.,  65.,  45.,  21.,  97.,   9.,  33.,
       101., 109., 105.,  61.,  41.,  73.,  49.,  57.,   5., 113.,  77.,
       121., 141., 117.,  81., 137.,  17.,  29., 129., 133., 125.,  25.,
        53.,  37.,  85.,  93.,   2.,  90.,  86.,  94.,  98.,  82.,  38.,
        18.,  78.,  74.,  42.,  70.,  14.,  66.,  46.,  62.,  10.,  22.,
        34., 102., 106., 110.,  58.,  50.,   6., 114., 142., 118., 122.,
       138.,  30., 134., 130.,  54.,  26., 126., 127., 123.,  27., 131.,
       135.,  31., 139., 119., 115., 143., 111., 107.,  23., 103.,  99.,
        35.,  95.,  19.,  91.,  87.,  83.,  39.,  15.,  79.,  75.,  71.,
        43.,  11.,  67.,  63.,  47.,   7.,  59.,  51.,  55.,   3.,   8.,
        52.,  56.,  48.,  60.,  44.,  64.,  68.,  40.,  72.,  76.,  80.,
        36.,  84.,  88.,  92.,  96.,  32., 100., 104., 108., 112., 116.,
       120., 124.,  24., 128., 132., 136., 140., 144.,  20.,  12., 147.,
       146., 148., 145.])

# Unit Coversion factor extraction

In [9]:
def load_conversion_factors(model_path):

    
    conv_file_path = os.path.join(model_path, 'initial.conv.sh')

    # Load conversion factors using cmc_parser - exactly as you showed
    conv_file = cp.conversion_file(conv_file_path)
        
    # Extract the conversion factors you demonstrated
    conversion_factors = {
        'mass_msun_mstar': conv_file.mass_msun_mstar,  # For converting masses
        'time_myr': conv_file.time_myr,                # For converting times 
        'length_parsec': conv_file.length_parsec,      # For converting radii
        'conv_obj': conv_file                          # Store full object for reference
    }
        
    return conversion_factors

# discover models

In [10]:

def parse_model_name(model_name: str):
    """
    Extract N, rv, rg and Z from directory names such as
    'N0.8rv0.75rg20Z0.002'  or  'N8e5rv0.75rg20z0.002_extra'.
    """
    # digits, dots, +, -, or scientific-notation letters
    num = r'([0-9.eE+-]+)'

    # ^ anchors at start; we don’t anchor at end so trailing text is allowed
    pattern = rf'^N{num}rv{num}rg{num}Z{num}'

    m = re.match(pattern, model_name, flags=re.IGNORECASE)
    if not m:
        print(f"Warning: Could not parse {model_name}")
        return None

    return {
        'N':  float(m.group(1)),
        'rv': float(m.group(2)),
        'rg': float(m.group(3)),
        'Z':  float(m.group(4)),
        'model_name': model_name
    }

In [11]:
def discover_models(data_directory):
    # Need to implement filtering
    model_dirs=[]
    for item in os.listdir(data_directory):
        item_path = os.path.join(data_directory, item)
        if os.path.isdir(item_path):
            parsed = parse_model_name(item)
            if parsed:
                parsed['path'] = item_path
                model_dirs.append(parsed)
        models_df = pd.DataFrame(model_dirs)
    return models_df

model_list = discover_models('data')
model_list

,N,rv,rg,Z,model_name,path
0,16.0,0.5,2.0,0.0020,N16rv0.5rg2.0Z0.002,data\N16rv0.5rg2.0Z0.002
1,16.0,0.5,2.0,0.0200,N16rv0.5rg2.0Z0.02,data\N16rv0.5rg2.0Z0.02
2,16.0,0.5,20.0,0.0020,N16rv0.5rg20.0Z0.002,data\N16rv0.5rg20.0Z0.002
3,16.0,0.5,20.0,0.0200,N16rv0.5rg20.0Z0.02,data\N16rv0.5rg20.0Z0.02
4,16.0,0.5,8.0,0.0020,N16rv0.5rg8.0Z0.002,data\N16rv0.5rg8.0Z0.002
...,...,...,...,...,...,...
140,8.0,4.0,20.0,0.0020,N8.0rv4.0rg20.0Z0.002,data\N8.0rv4.0rg20.0Z0.002
141,8.0,4.0,20.0,0.0200,N8.0rv4.0rg20.0Z0.02,data\N8.0rv4.0rg20.0Z0.02
142,8.0,4.0,8.0,0.0002,N8.0rv4.0rg8.0Z0.0002,data\N8.0rv4.0rg8.0Z0.0002
143,8.0,4.0,8.0,0.0020,N8.0rv4.0rg8.0Z0.002,data\N8.0rv4.0rg8.0Z0.002


# check each bh against merger file

In [12]:
def add_bbh_merger_column(all_bh, all_bbh_merge):
    bbh_merger_list = []
    for idx, row in all_bh.iterrows():
        bh_id = row['ID']
        bh_time = row['time']
        rows_with_id = all_bbh_merge[
            (all_bbh_merge['id1'] == bh_id) |
            (all_bbh_merge['id2'] == bh_id) |
            (all_bbh_merge['id3'] == bh_id)
        ]
        if rows_with_id.empty:
            bbh_merger_list.append(0)
        else:
            min_merger_time = rows_with_id['merger_time'].min()
            if min_merger_time < bh_time:
                bbh_merger_list.append(-1)
            else:
                bbh_merger_list.append(1)
    all_bh['bbh_merger'] = bbh_merger_list
    return all_bh

In [13]:
ag_bh_merge


,model_num,rv,rg,Z,N,merger_time,id1,id2,m1,m2,...,v_kick,v_esc,merger_channel,id3,m3,spin3,a_final,e_final,e_10hz,model_path
0,149.0,2.0,20.0,0.0020,1.0,2144.929793,62090.0,1934343.0,66.97210,72.4762,...,454.037000,129.4600,2.0,62090.0,132.6500,0.682164,0.008331,0.043318,-1.0,data\N1.0rv2.0rg20.0Z0.002
1,149.0,2.0,20.0,0.0020,1.0,2131.156995,7338925.0,1533922.0,16.43480,31.3124,...,150.523000,130.1470,3.0,12033922.0,45.8084,0.632367,0.000128,0.843304,-1.0,data\N1.0rv2.0rg20.0Z0.002
2,149.0,2.0,20.0,0.0020,1.0,2121.769751,10658409.0,92882.0,41.10010,42.0878,...,6.832950,139.8070,5.0,10658409.0,79.1733,0.686383,1.549980,0.999858,-1.0,data\N1.0rv2.0rg20.0Z0.002
3,149.0,2.0,20.0,0.0020,1.0,2113.954663,10720103.0,13016234.0,23.43160,30.0190,...,69.119300,135.5760,3.0,10720105.0,50.9368,0.678127,0.000145,0.821159,-1.0,data\N1.0rv2.0rg20.0Z0.002
4,149.0,2.0,20.0,0.0020,1.0,2113.701948,7209453.0,11690997.0,28.61570,36.9853,...,71.419400,134.4490,5.0,7209453.0,62.5217,0.677527,0.575687,0.999752,-1.0,data\N1.0rv2.0rg20.0Z0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14343,147.0,1.0,20.0,0.0200,32.0,716.737634,1061758.0,2577018.0,14.93740,14.9827,...,0.872944,93.4632,2.0,1061758.0,28.4758,0.686459,0.002783,0.218032,-1.0,data\N32rv1.0rg20.0Z0.02
14344,147.0,1.0,20.0,0.0200,32.0,406.007572,1749200.0,3954687.0,7.63013,12.9594,...,132.329000,112.0790,2.0,1749200.0,19.7060,0.649347,0.378760,0.999481,-1.0,data\N32rv1.0rg20.0Z0.02
14345,147.0,1.0,20.0,0.0200,32.0,405.472287,2514485.0,5593935.0,28.22760,14.8388,...,514.010000,116.0560,2.0,2514485.0,41.6730,0.503973,0.103354,0.992652,-1.0,data\N32rv1.0rg20.0Z0.02
14346,147.0,1.0,20.0,0.0200,32.0,461.040435,1785753.0,3558028.0,14.89870,14.2147,...,13.511400,105.6060,2.0,1785753.0,27.7094,0.686158,0.003220,0.422040,-1.0,data\N32rv1.0rg20.0Z0.02


# Full individual model processing

In [14]:
def add_bbh_merger_column(all_bh, all_bbh_merge):
    bbh_merger_list = []
    for idx, row in all_bh.iterrows():
        bh_id = row['ID']
        bh_time = row['time']
        rows_with_id = all_bbh_merge[
            (all_bbh_merge['id1'] == bh_id) |
            (all_bbh_merge['id2'] == bh_id) |
            (all_bbh_merge['id3'] == bh_id)
        ]
        if rows_with_id.empty:
            bbh_merger_list.append(0)
        else:
            min_merger_time = rows_with_id['merger_time'].min()
            if min_merger_time < bh_time:
                bbh_merger_list.append(-1)
            else:
                bbh_merger_list.append(1)
    all_bh['bbh_merger'] = bbh_merger_list
    return all_bh


In [21]:
def full_model(model_path,ag_bh_merge,latest=True):
    #getting the original frames
    all_bh=parse_bhformation_dat(model_path)
    all_col= parse_collision_log(model_path)
    all_merge=parse_merger_file(model_path)

    # getting the bbh merger frame
    all_bbh_merge = ag_bh_merge[ag_bh_merge['model_path']==model_path]

    # adding bbh merger indicator 
    all_bh = add_bbh_merger_column(all_bh, all_bbh_merge)


    # unit conversion
    conv_factors=load_conversion_factors(model_path)
    myr = conv_factors['time_myr']
    parsec = conv_factors['length_parsec']
    msun = conv_factors['mass_msun_mstar']

    #colision units
    all_col['time']=all_col['time']*myr# convert to MYr
    all_col['r']=all_col['r']*parsec # convert to Parsec
    
    #all bh units
    all_bh['time']=all_bh['time']*myr# convert to MYr
    all_bh['r']=all_bh['r']*parsec # convert to Parsec
    
    #all merge units
    all_merge['time']=all_merge['time']*myr# convert to MYr
    all_merge['r']=all_merge['r']*parsec # convert to Parsec

    #dropping all 0 ids
    all_bh = all_bh[all_bh['ID'] != 0]
    all_col = all_col[all_col['idm'] != 0]
    all_merge = all_merge[all_merge['id_rem'] != 0]

    #collision mass features 
    all_col['smaller_mass'] = all_col[['m1', 'm2']].min(axis=1)
    all_col['larger_mass'] = all_col[['m1', 'm2']].max(axis=1)
    all_col['mass_ratio'] = all_col['smaller_mass'] / all_col['larger_mass']
    # Collision and Merge BHs 
    col_bh = pd.merge(all_col,all_bh,left_on='idm',right_on='ID',how='inner')
    merge_bh = pd.merge(all_merge,all_bh,left_on='id_rem',right_on='ID',how='inner')

    # model path tagging
    all_bh['model'] = model_path
    all_col['model'] = model_path
    all_merge['model'] = model_path
    col_bh['model'] = model_path
    merge_bh['model'] = model_path
    
    #latest event merger fix

    if latest == True:
        filtered_df = merge_bh[merge_bh['time_y'] >= merge_bh['time_x']]

        merge_bh = filtered_df.loc[filtered_df.groupby('ID')['time_x'].idxmax()]


    #latest event col fix 9im iffy about this
    if latest == True:
        result_rows = []
        for id_val, id_group in col_bh.groupby('ID'):
        
            # Step 2: For each ID, group by time_y
            for time_y_val, time_y_group in id_group.groupby('time_y'):
            
                # Step 3: Drop rows where time_x > time_y
                valid_rows = time_y_group[time_y_group['time_x'] <= time_y_val]
            
                # Step 4: If valid rows exist, take the one with latest time_x
                if not valid_rows.empty:
                    latest_row = valid_rows.loc[valid_rows['time_x'].idxmax()]
                    result_rows.append(latest_row)
    col_bh = pd.DataFrame(result_rows)



    
    #output
    all_frames = {'all_bh':all_bh, 'col':all_col, 'merge':all_merge, 'col_bh':col_bh, 'merge_bh':merge_bh, 'all_bbh_merge':all_bbh_merge}
    return all_frames


# aggregating models 

In [16]:
all_paths=model_list['path'] 
# Initialize empty lists to collect DataFrames for each key
all_bh_list = []
merge_list = []
col_list = []
merge_bh_list = []
col_bh_list = []

# Loop through all paths and collect DataFrames
for path in all_paths:
    model = full_model(path,ag_bh_merge)
    all_bh_list.append(model['all_bh'])
    merge_list.append(model['merge'])
    col_list.append(model['col'])
    merge_bh_list.append(model['merge_bh'])
    col_bh_list.append(model['col_bh'])

# Now concatenate each list of DataFrames into single DataFrames
ag_all_bh = pd.concat(all_bh_list, ignore_index=True)
ag_merge = pd.concat(merge_list, ignore_index=True)
ag_col = pd.concat(col_list, ignore_index=True)
ag_merge_bh = pd.concat(merge_bh_list, ignore_index=True)
ag_col_bh = pd.concat(col_bh_list, ignore_index=True)

KeyboardInterrupt: 

# all data 

In [ ]:
ag_all_bh

In [ ]:
ag_merge

In [ ]:
ag_col

In [ ]:
ag_merge_bh

In [ ]:
ag_col_bh

# testing

## model list + merger list 


In [ ]:
model_list

In [ ]:
x = pd.DataFrame()

x['path']= ag_bh_merge['model_path'].unique()
x

In [ ]:
pd.merge(left=model_list,right=x,left_on=model_list['path'],right_on=x['path'].unique(),how='inner' )

In [ ]:
# Elements in x['path'] but NOT in model_list['path']
x_not_in_model = x[~x['path'].isin(model_list['path'])]

# Elements in model_list['path'] but NOT in x['path']  
model_not_in_x = model_list[~model_list['path'].isin(x['path'])]

# Just the path values as lists
x_only_paths = x_not_in_model['path'].tolist()
model_only_paths = model_not_in_x['path'].tolist()

# print("Paths in x but not in model_list:", x_only_paths)
# print("Paths in model_list but not in x:", model_only_paths)
x_only_paths

In [ ]:
print(model_only_paths)

## testing full model method


In [ ]:
ctest=full_model(model_list['path'][0],ag_bh_merge)['col_bh']
ctest

In [ ]:
btest=full_model(model_list['path'][0],ag_bh_merge)['all_bh']
btest

In [ ]:
i=0
for idx, row in btest.iterrows():
    if i==0:
        print(idx)
        print(row['time'])
    i=i+1

In [ ]:
x = add_bbh_merger_column(btest, ag_bh_merge)
x

## testing rv filters

In [20]:
def process_models_by_rv(model_list, ag_bh_merge, rv_min=0, rv_max=100):
    # Filter model_list based on rv min and max
    filtered_models = model_list[(model_list['rv'] >= rv_min) & (model_list['rv'] <= rv_max)]
    all_paths = filtered_models['path']

    # Initialize empty lists to collect DataFrames for each key
    all_bh_list = []
    merge_list = []
    col_list = []
    merge_bh_list = []
    col_bh_list = []

    # Loop through all filtered paths and collect DataFrames
    for path in all_paths:
        model = full_model(path, ag_bh_merge)
        all_bh_list.append(model['all_bh'])
        merge_list.append(model['merge'])
        col_list.append(model['col'])
        merge_bh_list.append(model['merge_bh'])
        col_bh_list.append(model['col_bh'])

    # Concatenate each list of DataFrames into single DataFrames with empty list handling
    ag_all_bh = pd.concat(all_bh_list, ignore_index=True) if all_bh_list else pd.DataFrame()
    ag_merge = pd.concat(merge_list, ignore_index=True) if merge_list else pd.DataFrame()
    ag_col = pd.concat(col_list, ignore_index=True) if col_list else pd.DataFrame()
    ag_merge_bh = pd.concat(merge_bh_list, ignore_index=True) if merge_bh_list else pd.DataFrame()
    ag_col_bh = pd.concat(col_bh_list, ignore_index=True) if col_bh_list else pd.DataFrame()


    # Create result dictionary with rv_max in the names
    rv_max_str = str(int(rv_max)) if rv_max == int(rv_max) else str(rv_max).replace('.', '_')

    result = {
        f'ag_all_bh_rv{rv_max_str}': ag_all_bh,
        f'ag_merge_rv{rv_max_str}': ag_merge,
        f'ag_col_rv{rv_max_str}': ag_col,
        f'ag_merge_bh_rv{rv_max_str}': ag_merge_bh,
        f'ag_col_bh_rv{rv_max_str}': ag_col_bh
    }

    return result


In [ ]:
f = process_models_by_rv(model_list, ag_bh_merge, rv_min=0, rv_max=1)


In [31]:
rv_list = [ 0.51 ,1.1  ,2.1 , 4.1 ]
col_percent=[]
c_per = 0.1111
for i in rv_list:
    c_per = 0.1111

    f = pd.Series(process_models_by_rv(model_list, ag_bh_merge, rv_min=0, rv_max=i))
    if f.iloc[0].shape[0] is not 0:
        c_per = f.iloc[4].shape[0]/f.iloc[0].shape[0]

    col_percent.append(c_per)
    print(f"rv = {i}:Percent {c_per*100}")

rv = 0.51:Percent 49.66348622861313
rv = 1.1:Percent 25.004347425812455
rv = 2.1:Percent 16.07445008460237
rv = 4.1:Percent 12.326943942133815


In [30]:
model_list

,N,rv,rg,Z,model_name,path
0,16.0,0.5,2.0,0.0020,N16rv0.5rg2.0Z0.002,data\N16rv0.5rg2.0Z0.002
1,16.0,0.5,2.0,0.0200,N16rv0.5rg2.0Z0.02,data\N16rv0.5rg2.0Z0.02
2,16.0,0.5,20.0,0.0020,N16rv0.5rg20.0Z0.002,data\N16rv0.5rg20.0Z0.002
3,16.0,0.5,20.0,0.0200,N16rv0.5rg20.0Z0.02,data\N16rv0.5rg20.0Z0.02
4,16.0,0.5,8.0,0.0020,N16rv0.5rg8.0Z0.002,data\N16rv0.5rg8.0Z0.002
...,...,...,...,...,...,...
140,8.0,4.0,20.0,0.0020,N8.0rv4.0rg20.0Z0.002,data\N8.0rv4.0rg20.0Z0.002
141,8.0,4.0,20.0,0.0200,N8.0rv4.0rg20.0Z0.02,data\N8.0rv4.0rg20.0Z0.02
142,8.0,4.0,8.0,0.0002,N8.0rv4.0rg8.0Z0.0002,data\N8.0rv4.0rg8.0Z0.0002
143,8.0,4.0,8.0,0.0020,N8.0rv4.0rg8.0Z0.002,data\N8.0rv4.0rg8.0Z0.002
